# Introduction to Planning for Self Driving Vehicles

In this notebook you are going to train your own ML policy to fully control an SDV. You will train your model using the Lyft Prediction Dataset and [L5Kit](https://github.com/woven-planet/l5kit).

**Before starting, please download the [Lyft L5 Prediction Dataset 2020](https://self-driving.lyft.com/level5/prediction/) and follow [the instructions](https://github.com/woven-planet/l5kit#download-the-datasets) to correctly organise it.**

The policy will be a deep neural network (DNN) which will be invoked by the SDV to obtain the next command to execute.

More in details, you will be working with a CNN architecture based on ResNet50.

![model](../../docs/images/planning/model.svg)


#### Inputs
The network will receive a Bird's-Eye-View (BEV) representation of the scene surrounding the SDV as the only input. This has been rasterised in a fixed grid image to comply with the CNN input. L5Kit is shipped with various rasterisers. Each one of them captures different aspects of the scene (e.g. lanes or satellite view).

This input representation is very similar to the one used in the [prediction competition](https://www.kaggle.com/c/lyft-motion-prediction-autonomous-vehicles/overview). Please refer to our [competition baseline notebook](../agent_motion_prediction/agent_motion_prediction.ipynb) and our [data format notebook](../visualisation/visualise_data.ipynb) if you want to learn more about it.

#### Outputs
The network outputs the driving signals required to fully control the SDV. In particular, this is a trajectory of XY and yaw displacements which can be used to move and steer the vehicle.

After enough training, your model will be able to drive an agent along a specific route. Among others, it will do lane-following while respecting traffic lights.

Let's now focus on how to train this model on the available data.

### Training using imitation learning
The model is trained using a technique called *imitation learning*. We feed examples of expert driving experiences to the model and expect it to take the same actions as the driver did in those episodes. Imitation Learning is a subfield of supervised learning, in which a model tries to learn a function f: X -> Y describing given input / output pairs - one prominent example of this is image classification.

This is also the same concept we use in our [motion prediction notebook](../agent_motion_prediction/agent_motion_prediction.ipynb), so feel free to check that out too.

##### Imitation learning limitations

Imitation Learning is powerful, but it has a strong limitation. It's not trivial for a trained model to generalise well on out-of-distribution data.

After training the model, we would like it to take full control and drive the AV in an autoregressive fashion (i.e. by following its own predictions).

During evaluation it's very easy for errors to compound and make the AV drift away from the original distribution. In fact, during training our model has seen only good examples of driving. In particular, this means **almost perfect midlane following**. However, even a small constant displacement during evaluation can accumulate enough error to lead the AV completely out of its distribution in a matter of seconds.

![drifting](../../docs/images/planning/drifting.svg)

This is a well known issue in SDV control and simulation discussed, among others, in [this article](https://ri.cmu.edu/pub_files/2010/5/Ross-AIStats10-paper.pdf).

# Adding perturbations to the mix

One of the simplest techniques to ensure a good generalisation is **data augmentation**, which exposes the network to different versions of the input and helps it to generalise better to out-of-distribution situations.

In our setting, we want to ensure that **our model can recover if it ends up slightly off the midlane it is following**.

Following [the noteworthy approach from Waymo](https://arxiv.org/pdf/1812.03079.pdf), we can enrich the training set with **online trajectory perturbations**. These perturbations are kinematically feasible and affect both starting angle and position. A new ground truth trajectory is then generated to link this new starting point with the original trajectory end point. These starting point will be slightly rotated and off the original midlane, and the new trajectory will teach the model how to recover from this situation.

![perturbation](../../docs/images/planning/perturb.svg)


In the following cell, we load the training data and leverage L5Kit to add these perturbations to our training set.
We also plot the same example with and without perturbation. During training, our model will see also those examples and learn how to recover from positional and angular offsets.

In [1]:
from tempfile import gettempdir
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from tqdm import tqdm

from l5kit.configs import load_config_data
from l5kit.data import LocalDataManager, ChunkedDataset
from l5kit.dataset import EgoDataset
from l5kit.rasterization import build_rasterizer
from l5kit.geometry import transform_points
from l5kit.visualization import TARGET_POINTS_COLOR, draw_trajectory
from l5kit.planning.rasterized.model import RasterizedPlanningModel
from l5kit.kinematic import AckermanPerturbation
from l5kit.random import GaussianRandomGenerator

import os

## Prepare data path and load cfg

By setting the `L5KIT_DATA_FOLDER` variable, we can point the script to the folder where the data lies.

Then, we load our config file with relative paths and other configurations (rasteriser, training params...).

In [2]:
!echo $PWD
# set env variable for data
os.environ["L5KIT_DATA_FOLDER"] = "/mnt/scratch/v_liuhaolan/l5kit_data"
dm = LocalDataManager(None)
# get config
cfg = load_config_data("./nmp-config-sample.yaml")

/mnt/home/v_liuhaolan/haolan/l5kit/nmp


In [3]:
perturb_prob = cfg["train_data_loader"]["perturb_probability"]

# rasterisation and perturbation
rasterizer = build_rasterizer(cfg, dm)
mean = np.array([0.0, 0.0, 0.0])  # lateral, longitudinal and angular
std = np.array([0.5, 1.5, np.pi / 6])


perturbation = AckermanPerturbation(
        random_offset_generator=GaussianRandomGenerator(mean=mean, std=std), perturb_prob=perturb_prob)

"""
# ===== INIT DATASET
train_zarr = ChunkedDataset(dm.require(cfg["train_data_loader"]["key"])).open()
train_dataset = EgoDataset(cfg, train_zarr, rasterizer, perturbation)

# plot same example with and without perturbation
for perturbation_value in [1, 0]:
    perturbation.perturb_prob = perturbation_value

    data_ego = train_dataset[1]
    im_ego = rasterizer.to_rgb(data_ego["image"].transpose(1, 2, 0))
    target_positions = transform_points(data_ego["target_positions"], data_ego["raster_from_agent"])
    draw_trajectory(im_ego, target_positions, TARGET_POINTS_COLOR)
    plt.imshow(im_ego)
    plt.axis('off')
    plt.show()
"""
# before leaving, ensure perturb_prob is correct
perturbation.perturb_prob = perturb_prob
print(perturb_prob)

0.1


In [4]:
#preprocessed_dir = "/mnt/scratch/v_liuhaolan/preprocessed"
#len(os.listdir(preprocessed_dir))

In [5]:
# ===== INIT DATASET
#from l5kit.dataset import CachedEgoDataset

#train_zarr = ChunkedDataset(dm.require(cfg["train_data_loader"]["key"])).open()
#train_dataset = CachedEgoDataset(cfg, train_zarr, rasterizer, preprocessed_path=preprocessed_dir,perturbation=perturbation)

# already in memory!

In [6]:
from l5kit.dataset import EgoDataset

train_zarr = ChunkedDataset(dm.require(cfg["train_data_loader"]["key"])).open()
train_dataset = EgoDataset(cfg, train_zarr, rasterizer, perturbation=perturbation)


In [7]:
# resnet should be too deep to train?
"""

model = RasterizedPlanningModel(
        model_arch="resnet50",
        num_input_channels=rasterizer.num_channels(),
        num_targets=3 * cfg["model_params"]["future_num_frames"],  # X, Y, Yaw * number of future states,
        weights_scaling= [1., 1., 1.],
        criterion=nn.L1Loss(reduction="none")
        )
"""
from model.models import RasterizedImitationModel
model = RasterizedImitationModel(
        model_arch="resnet50",
        num_input_channels=rasterizer.num_channels(),
        num_targets=3 * cfg["model_params"]["future_num_frames"],  # X, Y, Yaw * number of future states,
        weights_scaling= [1., 1., 1.],
        criterion=nn.L1Loss(reduction="none")
        )



print(model)



/mnt/home/v_liuhaolan/haolan/l5kit/nmp/model/models.py:40: UserWarning: There is no pre-trained model with num_in_channels != 3, first layer will be reset
  warnings.warn("There is no pre-trained model with num_in_channels != 3, first layer will be reset")


RasterizedImitationModel(
  (criterion): L1Loss()
  (model): ResNet(
    (conv1): Conv2d(5, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
     

In [8]:
print(model.model.avgpool)

AdaptiveAvgPool2d(output_size=(1, 1))


# Prepare for training
Our `EgoDataset` inherits from PyTorch `Dataset`; so we can use it inside a `Dataloader` to enable multi-processing.

In [10]:
train_cfg = cfg["train_data_loader"]
print(train_cfg["batch_size"])
train_dataloader = DataLoader(train_dataset, shuffle=train_cfg["shuffle"], batch_size=train_cfg["batch_size"], 
                             num_workers=train_cfg["num_workers"])
#device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
device = "cuda:0"
model = model.to(device)
#optimizer = optim.Adam(model.parameters(), lr=1e-3)

print(train_dataset)

64
+------------+------------+------------+---------------+-----------------+----------------------+----------------------+----------------------+---------------------+
| Num Scenes | Num Frames | Num Agents | Num TR lights | Total Time (hr) | Avg Frames per Scene | Avg Agents per Frame | Avg Scene Time (sec) | Avg Frame frequency |
+------------+------------+------------+---------------+-----------------+----------------------+----------------------+----------------------+---------------------+
|    100     |   24838    |  1893736   |     316008    |       0.69      |        248.38        |        76.24         |        24.83         |        10.00        |
+------------+------------+------------+---------------+-----------------+----------------------+----------------------+----------------------+---------------------+


In [11]:
# ===== INIT DATASET
"""
from l5kit.dataset import CachedEgoDataset

val_preprocessed_dir = "/mnt/scratch/v_liuhaolan/val_preprocessed"
len(os.listdir(val_preprocessed_dir))

val_zarr = ChunkedDataset(dm.require(cfg["val_data_loader"]["key"])).open()
val_dataset = CachedEgoDataset(cfg, val_zarr, rasterizer, preprocessed_path=val_preprocessed_dir,perturbation=perturbation)
"""
from l5kit.dataset import EgoDataset
#from l5kit.evaluation import create_chopped_dataset

val_zarr = ChunkedDataset(dm.require(cfg["val_data_loader"]["key"])).open()
val_dataset = EgoDataset(cfg, val_zarr, rasterizer, perturbation=perturbation)


In [12]:
val_cfg = cfg["val_data_loader"]
val_dataloader = DataLoader(val_dataset, shuffle=val_cfg["shuffle"], batch_size=val_cfg["batch_size"], 
                             num_workers=val_cfg["num_workers"])

print(val_dataset)

+------------+------------+------------+---------------+-----------------+----------------------+----------------------+----------------------+---------------------+
| Num Scenes | Num Frames | Num Agents | Num TR lights | Total Time (hr) | Avg Frames per Scene | Avg Agents per Frame | Avg Scene Time (sec) | Avg Frame frequency |
+------------+------------+------------+---------------+-----------------+----------------------+----------------------+----------------------+---------------------+
|    100     |   24838    |  1893736   |     316008    |       0.69      |        248.38        |        76.24         |        24.83         |        10.00        |
+------------+------------+------------+---------------+-----------------+----------------------+----------------------+----------------------+---------------------+


In [13]:
#sample_data["negative_positions_pixels"]
len(val_dataloader)

518

In [14]:
#transform_points(sampled_positions_maps, data["raster_from_agent"])
"""

pixel_size = (cfg['raster_params']['pixel_size'][0])

# TODO: runtime checking when the sampling trajectory will move out of the rasterized image
def max_margin_loss(batch_negative_index, batch_gt_index, cost_map):
    # calculate the costs aggregated with regard to time.
    batch_size = 64
    traj_size = 12
    loss = 0
    for i in range(batch_size):
        each_loss = 0
        diff = batch_gt_index[i] - batch_negative_index[i]
        diff = torch.sum(torch.pow(torch.sum(torch.pow(diff, 2),dim=1),0.5), dim=0).item()
#        print(diff*pixel_size)
        
        for j in range(traj_size):
            time_step = cost_map[i][batch_gt_index[i][j][0]][batch_gt_index[i][j][1]] - cost_map[i][batch_negative_index[i][j][0]][batch_negative_index[i][j][1]] + diff*pixel_size
#            print(time_step)
            if time_step > 0:
                each_loss = each_loss + (time_step)
        loss = loss + each_loss
    return loss / batch_size

import time
start = time.time()

#print(sample_data)
#data_batch["negative_positions_pixels"], data_batch["target_positions_pixels"], outputs["cost_map"]

r = max_margin_loss(sample_data["negative_positions_pixels"], sample_data["target_positions_pixels"], outputs["cost_map"])
print(r)
print("time: {} s".format(time.time()-start))
"""
# cpu 0.083
# gpu: 0.171 needs to be vectorized!

'\n\npixel_size = (cfg[\'raster_params\'][\'pixel_size\'][0])\n\n# TODO: runtime checking when the sampling trajectory will move out of the rasterized image\ndef max_margin_loss(batch_negative_index, batch_gt_index, cost_map):\n    # calculate the costs aggregated with regard to time.\n    batch_size = 64\n    traj_size = 12\n    loss = 0\n    for i in range(batch_size):\n        each_loss = 0\n        diff = batch_gt_index[i] - batch_negative_index[i]\n        diff = torch.sum(torch.pow(torch.sum(torch.pow(diff, 2),dim=1),0.5), dim=0).item()\n#        print(diff*pixel_size)\n        \n        for j in range(traj_size):\n            time_step = cost_map[i][batch_gt_index[i][j][0]][batch_gt_index[i][j][1]] - cost_map[i][batch_negative_index[i][j][0]][batch_negative_index[i][j][1]] + diff*pixel_size\n#            print(time_step)\n            if time_step > 0:\n                each_loss = each_loss + (time_step)\n        loss = loss + each_loss\n    return loss / batch_size\n\nimport tim

# Training loop
Here, we purposely include a barebone training loop. Clearly, many more components can be added to enrich logging and improve performance. Still, the sheer size of our dataset ensures that a reasonable performance can be obtained even with this simple loop.

In [16]:
#tr_it = iter(train_dataloader)
#progress_bar = tqdm(range(cfg["train_params"]["max_num_steps"]))

optimizer = optim.Adam(model.parameters(), lr=1e-4)
epoch_num = 8
losses_train1 = []

max_val_loss = 100000

for epochs in range(epoch_num):
    
    iter_bar = tqdm(train_dataloader, desc='Iter (loss=X.XXX)')
    
    model.train()
    for step,batch in enumerate(iter_bar):
        # Forward pass
        batch = {k: v.to(device) for k, v in batch.items()}
        result = model(batch)
        loss = result["loss"]
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        
#        if step%10000 == 0:
#            print("model save")
#            torch.save(model.state_dict(),"./planning_model.pt")

        losses_train1.append(loss.item())
        loss_v = np.round(loss.item(),3)
        loss_average = np.round(np.mean(losses_train1) ,3)
        
        motion_loss = result["motion_loss"]
        motion_loss = np.round(motion_loss.item(),3)
        target_loss = result["target_loss"]
        target_loss = np.round(target_loss.item(),3)
#        iter_bar.set_description(f"loss: {(loss_v)} loss(avg): {loss_average}")
        iter_bar.set_description(f"loss: {(loss_v)} motion_loss: {motion_loss} target_loss: {target_loss}")

    # write a loop in evaluation dataset
    model.eval()  
    loss_val = 0
    
    cnt = 10
    for step,batch in enumerate(val_dataloader):
        # inference pass
        batch = {k: v.to(device) for k, v in batch.items()}
        result = model(batch)
        loss = result["loss"]
        
#        if step%10000 == 0:
#            print("model save")
#            torch.save(model.state_dict(),"./planning_model.pt")
        
        loss_val += loss.item()
        
        if step >= cnt:
            break
    loss_average = loss_val/cnt
    print("val loss: {}".format(loss_average))
#    if loss_average < max_val_loss:
#        max_val_loss = loss_average
#        torch.save(model, "./planning_model1.pt")
    
        
# 2.52 it/s read from memory

loss: 0.986 motion_loss: 0.701 target_loss: 2.848: 100%|█| 389/389 [04:21<00:00,


val loss: 0.7330867188051343


loss: 0.322 motion_loss: 0.193 target_loss: 1.282: 100%|█| 389/389 [04:27<00:00,


val loss: 0.4727870259433985


loss: 0.259 motion_loss: 0.16 target_loss: 0.994: 100%|█| 389/389 [04:18<00:00, 


...

val loss: 0.18773239403963088


loss: 0.711 motion_loss: 0.378 target_loss: 3.327: 100%|█| 389/389 [04:18<00:00,


val loss: 0.19600312151014804


loss: 0.11 motion_loss: 0.033 target_loss: 0.772: 100%|█| 389/389 [04:19<00:00, 


val loss: 0.191843187995255


In [ ]:
print(loss_average)

In [ ]:
#tr_it = iter(train_dataloader)
#progress_bar = tqdm(range(cfg["train_params"]["max_num_steps"]))

#model = torch.load(""./planning_model1.pt")
optimizer = optim.Adam(model.parameters(), lr=1e-5)
epoch_num = 4
losses_train1 = []


max_val_loss = 100000

for epochs in range(epoch_num):
    
    iter_bar = tqdm(train_dataloader, desc='Iter (loss=X.XXX)')
    
    model.train()
    for step,batch in enumerate(iter_bar):
        # Forward pass
        batch = {k: v.to(device) for k, v in batch.items()}
        result = model(batch)
        loss = result["loss"]
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        
#        if step%10000 == 0:
#            print("model save")
#            torch.save(model.state_dict(),"./planning_model.pt")

        losses_train1.append(loss.item())
        loss_v = np.round(loss.item(),6)
        loss_average = np.round(np.mean(losses_train1) ,6)
        iter_bar.set_description(f"loss: {(loss_v)} loss(avg): {loss_average}")

    # write a loop in evaluation dataset
    model.eval()  
    loss_val = 0
    for step,batch in enumerate(val_dataloader):
        # inference pass
        batch = {k: v.to(device) for k, v in batch.items()}
        result = model(batch)
        loss = result["loss"]
        
#        if step%10000 == 0:
#            print("model save")
#            torch.save(model.state_dict(),"./planning_model.pt")
        
        loss_val += loss.item()
    loss_average = loss_val/len(val_dataloader)
    print("val loss: {}".format(loss_average))
    if loss_average < max_val_loss:
        max_val_loss = loss_average
        torch.save(model, "./planning_model2.pt")
    
        

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=1e-6)
epoch_num = 20
losses_train3 = []

for epochs in range(epoch_num):
    
    iter_bar = tqdm(train_dataloader, desc='Iter (loss=X.XXX)')
    for step,batch in enumerate(iter_bar):
        # Forward pass
        batch = {k: v.to(device) for k, v in batch.items()}
        result = model(batch)
        loss = result["loss"]
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
#        if step%10000 == 0:
#            print("model save")
#            torch.save(model.state_dict(),"./planning_model.pt")

        losses_train3.append(loss.item())
        iter_bar.set_description(f"loss: {loss.item()} loss(avg): {np.mean(losses_train3)}")


### Plot the train loss curve
We can plot the train loss against the iterations (batch-wise) to check if our model has converged.

In [ ]:
plt.plot(np.arange(len(losses_train2)), losses_train2, label="train loss")
plt.legend()
plt.show()

In [18]:
#torch.save(model.state_dict(),"./planning_model.pt")
torch.save(model.state_dict(), "./planning_model_tnt.pt")

In [ ]:
sample_data["image"].shape

In [ ]:
# visualize the costmap
sample_data = next(iter(train_dataloader))
for i in range(10):
    sample_data = next(iter(train_dataloader))
sample_data = {k: v.to(device) for k, v in sample_data.items()}
outputs = model(sample_data)

print(outputs["cost_map"].shape)

# visualize the cost map!
idx = 2
cost_image = outputs["cost_map"][idx].reshape(112, 112)

#print(cost_image)


from torchvision import transforms
 
# define custom transform
# here we are using our calculated
# mean & std

# get normalized image
#img_normalized = transform_norm(cost_image.numpy())
 
# convert this image to numpy array
img_normalized = np.array(cost_image.cpu().detach())
 
# transpose from shape of (3,,) to shape of (,,3)
#img_normalized = img_normalized.transpose(1, 0)
 
# display the normalized image
plt.imshow(img_normalized, cmap='gray')
plt.xticks([])
plt.yticks([])

"""
im = np.array(sample_data["image"][idx].cpu().detach()).transpose(1, 2, 0)
im = train_dataset.rasterizer.to_rgb(im)
target_positions_pixels = transform_points(sample_data["target_positions"][idx].cpu().numpy(), sample_data["raster_from_agent"][idx].cpu().numpy())
draw_trajectory(im, target_positions_pixels, TARGET_POINTS_COLOR, yaws=sample_data["target_yaws"][idx].cpu().numpy())
"""


In [ ]:
im = np.array(sample_data["image"][idx].cpu().detach()).transpose(1, 2, 0)
im = train_dataset.rasterizer.to_rgb(im)
#target_positions_pixels = transform_points(sample_data["target_positions"][idx].cpu().numpy(), sample_data["raster_from_agent"][idx].cpu().numpy())
#draw_trajectory(im, target_positions_pixels, TARGET_POINTS_COLOR, yaws=sample_data["target_yaws"][idx].cpu().numpy())
plt.imshow(im)
plt.show()



In [ ]:
print(img_normalized[55][65])


# Store the model

Let's store the model as a torchscript. This format allows us to re-load the model and weights without requiring the class definition later.

**Take note of the path, you will use it later to evaluate your planning model!**

In [ ]:
to_save = torch.jit.script(model.cpu())
path_to_save = f"{gettempdir()}/planning_model.pt"
to_save.save(path_to_save)
print(f"MODEL STORED at {path_to_save}")

# Congratulations in training your first ML policy for planning!
### What's Next

Now that your model is trained and safely stored, you can evaluate how it performs in two very different situations using our dedicated notebooks:

### [Open-loop evaluation](./open_loop_test.ipynb)
In this setting the model **is not controlling the AV**, and predictions are used to compute metrics only.

### [Closed-loop evaluation](./closed_loop_test.ipynb)
In this setting the model **is in full control of the AV** future movements.

## Pre-trained models
we provide a collection of pre-trained models for the planning task:
- [model](https://lyft-l5-datasets-public.s3-us-west-2.amazonaws.com/models/planning_models/planning_model_20201208.pt) trained on  train.zarr for 15 epochs;
- [model](https://lyft-l5-datasets-public.s3-us-west-2.amazonaws.com/models/planning_models/planning_model_20201208_early.pt) trained on train.zarr for 2 epochs;
- [model](https://lyft-l5-datasets-public.s3-us-west-2.amazonaws.com/models/planning_models/planning_model_20201208_nopt.pt) trained on train.zarr with perturbations disabled for 15 epochs;
- [model](https://lyft-l5-datasets-public.s3-us-west-2.amazonaws.com/models/planning_models/planning_model_20201208_nopt_early.pt) trained on train.zarr with perturbations disabled for 2 epochs;

We include two partially trained models to emphasise the important role of perturbations during training, especially during the first stage of training.

To use one of the models simply download the corresponding `.pt` file and load it in the evaluation notebooks.